In [4]:
import os
import pandas as pd

In [5]:
global_north = ['Australia','England','New Zealand','South Africa']
global_south = ['India','Pakistan','Bangladesh','West Indies']

In [6]:
def calculate_gn_gs(path,shot_type):
    runs = os.listdir(path)
    df_gn_total = None
    df_gs_total = None
    firstTime = True
    
    for run in ['run_1','run_2','run_3']:
        shots = os.listdir(path+'/'+run)
        for shot in ['1_shot','2_shot']:
            if shot == shot_type:
                cur_path = path+'/'+run+'/'+shot
                df = pd.read_csv(cur_path+'/all_accuracy.csv')
                df_gn = df[df['country'].isin(global_north)]
                df_gs = df[df['country'].isin(global_south)]
                
                if firstTime:
                    df_gn_total = df_gn
                    df_gs_total = df_gs
                    firstTime = False
                else:
                    df_gn_total.iloc[:, 1:] = df_gn_total.iloc[:, 1:] + df_gn.iloc[:, 1:]
                    df_gs_total.iloc[:, 1:] = df_gs_total.iloc[:, 1:] + df_gs.iloc[:, 1:]
#                 print(df_gn.head())
    return df_gn_total,df_gs_total

In [7]:
def global_north_south(df_gn_total,df_gs_total,gender,shot_type):
    male_gn = pd.DataFrame()
    male_gn['region'] = None
    lis = ['global_north']
    for label, value in df_gn_total.items():
        if label.split('-')[0] == gender:
            male_gn["-".join(label.split('-')[1:])] = None
            lis.append(value)
    male_gn.at[len(male_gn.index)] = lis

    male_gs = pd.DataFrame()
    male_gs['region'] = None
    lis = ['gloabal_south']
    for label, value in df_gs_total.items():
        if label.split('-')[0] == gender:
            male_gs["-".join(label.split('-')[1:])] = None
            lis.append(value)
    male_gs.at[len(male_gs.index)] = lis
    
    male_gn = male_gn.append(male_gs)
    male_gn.to_csv(path+'/'+shot_type+"_"+gender+'.csv',index = False)
    return male_gn

In [17]:
API_NAME = "adverserial" ## For Fine-tuning setup use fine-tuning image type
shot_type = "1_shot"

path = "./../Test Data v3/plots/"+API_NAME

df_gn_total,df_gs_total = calculate_gn_gs(path,shot_type)

df_gn_total.iloc[:, 1:] = df_gn_total.iloc[:, 1:]/3
df_gn_total = df_gn_total.select_dtypes(include='number').sum()/4
df_gn_total = df_gn_total.round(2)

df_gs_total.iloc[:, 1:] = df_gs_total.iloc[:, 1:]/3
df_gs_total = df_gs_total.select_dtypes(include='number').sum()/4
df_gs_total = df_gs_total.round(2)

for gender in ['overall','male','female']:
    print(global_north_south(df_gn_total,df_gs_total,gender,shot_type))

          region ORIGINAL GREYSCALE RGB-0.3 RGB-0.5 SPREAD MASKED
0   global_north    91.67     90.97   92.36   87.08  91.81  89.44
0  gloabal_south    90.55      93.2   87.08   76.39  88.75  85.83
          region ORIGINAL GREYSCALE RGB-0.3 RGB-0.5 SPREAD MASKED
0   global_north     87.5     86.46   91.04   93.75  87.71   90.0
0  gloabal_south    93.33     94.17   94.58   96.04  90.21  91.25
          region ORIGINAL GREYSCALE RGB-0.3 RGB-0.5 SPREAD MASKED
0   global_north    100.0     100.0    95.0   73.75  100.0  88.33
0  gloabal_south     85.0     91.25   72.08   37.08  85.83   75.0


## Analysis on Original-models

In [143]:
def calculate_gn_gs_org(path):
    
    df = pd.read_csv(path+'/all_accuracy.csv')
    df_gn = df[df['country'].isin(global_north)]
    df_gs = df[df['country'].isin(global_south)]
    
    return df_gn,df_gs

In [144]:
API_NAME = "triplet_loss_full_mf_82_100_epochs"

path = "./../Test Data v3/plots/"+API_NAME

df_gn_total,df_gs_total = calculate_gn_gs_org(path)

df_gn_total.iloc[:, 1:] = df_gn_total.iloc[:, 1:]
df_gn_total = df_gn_total.select_dtypes(include='number').sum()/4
df_gn_total = df_gn_total.round(2)

df_gs_total.iloc[:, 1:] = df_gs_total.iloc[:, 1:]
df_gs_total = df_gs_total.select_dtypes(include='number').sum()/4
df_gs_total = df_gs_total.round(2)

for gender in ['overall','male','female']:
    print(global_north_south(df_gn_total,df_gs_total,gender,"region_wise_"))

          region ORIGINAL GREYSCALE RGB-0.3 RGB-0.5 SPREAD MASKED
0   global_north    84.58      80.0   84.17   81.66  82.92  79.58
0  gloabal_south    86.67     77.08   83.75   83.33  85.42  78.33
          region ORIGINAL GREYSCALE RGB-0.3 RGB-0.5 SPREAD MASKED
0   global_north     82.5     81.25   81.88   78.75  81.25  93.12
0  gloabal_south     90.0     81.88    87.5   86.88  88.12  93.75
          region ORIGINAL GREYSCALE RGB-0.3 RGB-0.5 SPREAD MASKED
0   global_north    88.75      77.5   88.75    87.5  86.25   52.5
0  gloabal_south     80.0      67.5   76.25   76.25   80.0   47.5
